# Pricing a European Call Option Version 2

----

#### John Stachurski (August 2024)

----

In this notebook we will accelerate our code for option pricing using different
libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Why is Pure Python Slow?

We saw that our Python code for option pricing was pretty slow.

In essence, this is because our loops were written in pure Python

Pure Python loops are not fast.

This has led some people to claim that Python is too slow for computational
economics.

These people are ~idiots~ misinformed -- so please ignore them.

Evidence: AI teams are solving optimization problems in $\mathbb R^d$ with $d >
$ 1 trillion using Pytorch / JAX

So I'm pretty sure we can use Python for computational economics.

But first let's try to understand the issues.

### Issue 1: Type Checking

Consider the following Python code

In [ ]:
x, y = 1, 2
x + y

This is integer addition, which is different from floating point addition

In [ ]:
x, y = 1.0, 2.0
x + y

Now consider this code

In [ ]:
x, y = 'foo', 'bar'
x + y

Notice that we use the same symbol `+` on each occasion.

The Python interpreter figures out the correct action by type checking:

In [ ]:
a, b = 'foo', 10
type(a)

In [ ]:
type(b)

But think of all the type checking in our option pricing function --- the
overhead is huge!!

In [ ]:
n, β, K = 10, 0.99, 100
μ, ρ, ν, S_0, h_0 = 0.0001, 0.01, 0.001, 10.0, 0.0
def compute_call_price_py(β=β,
                           μ=μ,
                           S_0=S_0,
                           h_0=h_0,
                           K=K,
                           n=n,
                           ρ=ρ,
                           ν=ν,
                           M=1_000_000,
                           seed=1234):
    np.random.seed(seed)

    s_0 = np.log(S_0)
    s_n = np.empty(M)

    for m in range(M):
        s, h = s_0, h_0
        for t in range(n):
            U, V = np.random.randn(2)
            s = s + μ + np.exp(h) * U
            h = ρ * h + ν * V
        s_n[m] = s

    S_n = np.exp(s_n)

    expectation = np.mean(np.maximum(S_n - K, 0))

    return β**n * expectation

### Issue 2:  Memory Management

Pure Python emphasizes flexibility and hence cannot attain maximal efficiency
vis-a-vis memory management.

For example,

In [ ]:
import sys
x = [1.0, 2.0]  
sys.getsizeof(x) * 8   # number of bits

### Issue 3:  Parallelization

There are opportunities to parallelize our code above -- divide it across
multiple workers.

This can't be done efficiently with pure Python but certainly can with the right
Python libraries.

## Vectorization

As a first pass at improving efficiency, here's a vectorized version where all paths are updated together.

We use NumPy to store and update each vector of share prices.

When we use NumPy, type-checking is done per-array, not per element!

In [ ]:
def compute_call_price_np(β=β,
                          μ=μ,
                          S_0=S_0,
                          h_0=h_0,
                          K=K,
                          n=n,
                          ρ=ρ,
                          ν=ν,
                          M=10_000_000,
                          seed=1234):
    np.random.seed(seed)
    s = np.full(M, np.log(S_0))
    h = np.full(M, h_0)
    for t in range(n):
        U, V = np.random.randn(M), np.random.randn(M)
        s = s + μ + np.exp(h) * U
        h = ρ * h + ν * V
    expectation = np.mean(np.maximum(np.exp(s) - K, 0))
        
    return β**n * expectation

Now computation of the option price is reasonably fast:

In [ ]:
%time compute_call_price_np()

But we can still do better...

## Numba Version

Let's try a Numba version.

This version uses a just-in-time (JIT) compiler to eliminate type-checking.

In [ ]:
import numba

@numba.jit()
def compute_call_price_numba(β=β,
                               μ=μ,
                               S_0=S_0,
                               h_0=h_0,
                               K=K,
                               n=n,
                               ρ=ρ,
                               ν=ν,
                               M=10_000_000,
                               seed=1234):
    np.random.seed(seed)
    s_0 = np.log(S_0)
    s_n = np.empty(M)
    for m in range(M):
        s, h = s_0, h_0
        for t in range(n):
            s = s + μ + np.exp(h) * np.random.randn()
            h = ρ * h + ν * np.random.randn()
        s_n[m] = s
    S_n = np.exp(s_n)
    expectation = np.mean(np.maximum(S_n - K, 0))

    return β**n * expectation

In [ ]:
%time compute_call_price_numba()

In [ ]:
%time compute_call_price_numba()

## Numba Plus Parallelization

The last version was only running on one core.

Next let's try a Numba version with parallelization.

In [ ]:
from numba import prange

@numba.jit(parallel=True)
def compute_call_price_numba_parallel(β=β,
                                      μ=μ,
                                      S_0=S_0,
                                      h_0=h_0,
                                      K=K,
                                      n=n,
                                      ρ=ρ,
                                      ν=ν,
                                      M=10_000_000,
                                      seed=1234):
    np.random.seed(seed)
    s_0 = np.log(S_0)
    s_n = np.empty(M)
    for m in prange(M):
        s, h = s_0, h_0
        for t in range(n):
            s = s + μ + np.exp(h) * np.random.randn()
            h = ρ * h + ν * np.random.randn()
        s_n[m] = s
    S_n = np.exp(s_n)
    expectation = np.mean(np.maximum(S_n - K, 0))

    return β**n * expectation

In [ ]:
%time compute_call_price_numba_parallel()

In [ ]:
%time compute_call_price_numba_parallel(M=100_000_000)

## JAX Version

We can do even better if we exploit a hardware accelerator such as a GPU.

Let's see if we have a GPU:

In [ ]:
!nvidia-smi

The following import is standard, replacing `import numpy as np`:

In [ ]:
import jax
import jax.numpy as jnp

### Simple JAX version

Let's start with a simple version that looks like the NumPy version.

In [ ]:
def compute_call_price_jax(β=β,
                           μ=μ,
                           S_0=S_0,
                           h_0=h_0,
                           K=K,
                           n=n,
                           ρ=ρ,
                           ν=ν,
                           M=10_000_000,
                           seed=1234):

    key=jax.random.PRNGKey(seed)
    s = jnp.full(M, np.log(S_0))
    h = jnp.full(M, h_0)
    for t in range(n):
        Z = jax.random.normal(key, (2, M))
        s = s + μ + jnp.exp(h) * Z[0, :]
        h = ρ * h + ν * Z[1, :]
        key = jax.random.fold_in(key, t)

    S = jnp.exp(s)
    expectation = jnp.mean(jnp.maximum(S - K, 0))
        
    return β**n * expectation

Let's run it once to compile it:

In [ ]:
%%time 
price = compute_call_price_jax()
print(price)

And now let's time it:

In [ ]:
%%time 
price = compute_call_price_jax()
print(price)

### Compiled JAX version

Let's take the simple JAX version above and compile the entire function.

In [ ]:
compute_call_price_jax_compiled = jax.jit(compute_call_price_jax, static_argnums=(5, 8, ))

We run once to compile.

In [ ]:
%%time 
price = compute_call_price_jax_compiled()
print(price)

And now let's time it.

In [ ]:
%%time 
price = compute_call_price_jax_compiled()
print(price)

Now we have a really big speed gain relative to NumPy.